In [1]:
import pandas as pd
from datetime import date
import cbpro
import numpy as np

import matplotlib.pyplot as plt

from portfolio import Asset

C:\Users\Artem\AppData\Local\Temp\ipykernel_28240\266322298.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
with open('CoinBaseAPIKey.key', 'r') as f:
    contents = f.readlines()
cb_key = contents[0].split(':')[-1].strip()
cb_secret = contents[1].split(':')[-1].strip()

In [3]:
#Download data
public_client = cbpro.PublicClient()
slots = [('2023-01-01', '2023-06-01'), ('2022-07-01', '2022-12-31'), ('2022-01-01', '2022-06-30'), ('2023-06-02', '2023-12-01')]
BTC = Asset('BTC')
ETH = Asset('ETH')
USDT = Asset('USDT')

for slot in slots:
    BTC_USD = public_client.get_product_historic_rates('BTC-USD', slot[0], slot[1], 86400)
    ETH_USD = public_client.get_product_historic_rates('ETH-USD', slot[0], slot[1], 86400)
    USDT_USD = public_client.get_product_historic_rates('USDT-USD', slot[0], slot[1], 86400)
    btc_data= pd.DataFrame(BTC_USD)
    btc_data.columns = [ 'date_time', 'low', 'high', 'open', 'close', 'volume' ]
    btc_data['date_time']= btc_data['date_time'].apply(lambda x: date.fromtimestamp(x))
    BTC.update_history(btc_data)

    ETH_data= pd.DataFrame(ETH_USD)
    ETH_data.columns = [ 'date_time', 'low', 'high', 'open', 'close', 'volume' ]
    ETH_data['date_time']= ETH_data['date_time'].apply(lambda x: date.fromtimestamp(x))
    ETH.update_history(ETH_data)

    USDT_data= pd.DataFrame(USDT_USD)
    USDT_data.columns = [ 'date_time', 'low', 'high', 'open', 'close', 'volume' ]
    USDT_data['date_time']= USDT_data['date_time'].apply(lambda x: date.fromtimestamp(x))
    USDT.update_history(USDT_data)

In [ ]:
class Portfolio:
    def __init__(self):
        idx  = pd.MultiIndex(levels=[[],[]],
                          codes=[[],[]],
                          names=[u'date_time', u'ticker'])
        my_columns = [u'change', u'note']
        self.transactions  = pd.DataFrame(index=idx, columns=my_columns)
        self.value = pd.DataFrame(columns = ['date_time', 'value'], index = pd.Index([], name='date_time'))


    def update_transactions(self, ticker:str, qty:float, transaction_date = date.today(), note = ''):
        self.transactions.loc[(transaction_date,ticker,),:] = [qty, note]

    def get_positions(self, on_date = date.today()):
        time_mask = (self.transactions['date_time'] <= on_date)
        positions = self.transactions[time_mask][['ticker', 'change']]\
                        .groupby(['ticker'])\
                        .agg({'change':'sum'})\
                        .reset_index()
        positions.columns = ['ticker', 'position_size']
        positions['ticker'] = positions['ticker'].astype(str)
        positions['position_value'] = positions['ticker'].apply(
                                                lambda x: Asset.asset_dict[x].price_on_date(on_date)
                                                )
        positions['position_value'] = positions['position_value']*positions['position_size']
        return positions

    def get_value(self, on_date = date.today()):
        self.update_value()
        mask = (self.value['date_time'] == on_date)
        return self.value[mask]['value']
    
    def update_value(self):
        self.value.drop(self.value.index, inplace=True)
        #generate a list of dates to update
        date_to_add = date.fromisoformat('2023-11-30')
        oldest_transaction = min(self.transactions['date_time'])
        while date_to_add >= oldest_transaction:
            composition_at_date = self.get_positions(date_to_add)
            value_to_add = composition_at_date.position_value.sum()
            df_to_add = pd.DataFrame({'value': value_to_add, 'date_time': date_to_add}, index=[0])
            self.value = pd.concat([self.value, df_to_add], ignore_index=True)
            date_to_add = date_to_add - timedelta(days=1)

In [4]:
my_portfolio = Portfolio()
my_portfolio.do_transaction(ticker='BTC', qty=1, transaction_date=date.fromisoformat('2022-01-01'))
my_portfolio.do_transaction(ticker='USDT', qty=40000, transaction_date=date.fromisoformat('2022-01-01'))

,,change,note
date_time,ticker,,


In [51]:

transactions


,,change,note
date_time,ticker,,
2024-04-18,ASD,1,qwe
